<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Data_Cleaning2(FJC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal effect of Patent Litigation Outcomes on Firm Financials


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [118]:
JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')

<ipython-input-118-caae5146eea8>:1: DtypeWarning: Columns (1,2,9,10,11,13,19,20,23,24,26,38,39,40,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')


In [119]:
#ISOLATING PANTENT AND TRADENARK INFRINGEMENT SUITES, AND ONLY ORIGINAL PROCEEDINGS
#Patents_1988_forwards = pd.DataFrame(JFC_1988_Present[JFC_1988_Present['NOS'].isin([830, 840])]) #only patent/trademark cases
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ORIGIN'] == 1] #only original filings
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ARBIT'].astype(str) == '-8'] #no formal arbitration courts

clean_JFC_patents = Patents_1988_forwards.copy()

#removing columns that aren't relavent or mostly empty
clean_JFC_patents.drop(columns=['ORIGIN','TITL', 'SECTION', 'SUBSECT', 'ARBIT',
                                'FILEJUDG', 'FILEMAG', 'MDLDOCK',  'TRANSDAT',
                                'TRANSOFF', 'TRANSDOC', 'TRANSORG', 'TDATEUSE',
                                'TRCLACT', 'TERMJUDG', 'TERMMAG', 'DJOINED',
                                'PRETRIAL', 'TRIBEGAN', 'TRIALEND', 'AMTREC',
                                'TRMARB', 'RESIDENC', 'PROSE', 'IFP'],
                                 inplace=True)

In [120]:
clean_JFC_patents.head()

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,NOS,JURY,CLASSACT,...,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD,TAPEYEAR
42,0,90,1,8203478,12/08/1982,12/01/1982,3,830,-8,-8,...,99999,CHUBB INTEG,NATL BANK WASH,02/10/1988,3,13,-8,-8,NaN,1988
47,0,90,1,8300001,01/03/1983,01/01/1983,3,830,-8,-8,...,88888,NEWMAN,MOSSINGHOFF,02/17/1988,8,9,0,2,NaN,1988
188,0,90,1,8403233,10/22/1984,10/01/1984,3,830,-8,-8,...,88888,CORP.COMMUNICATIONS,CAPITOL,06/01/1988,2,0,-8,-8,NaN,1988
241,0,90,1,8403720,12/06/1984,12/01/1984,3,840,-8,-8,...,11001,WASH CHIROPTC HLTH,CHIROPTC INC,10/01/1987,3,3,-8,-8,NaN,1988
447,0,90,1,8501100,04/08/1985,04/01/1985,3,840,-8,-8,...,88888,CHARLES ETAL,GALERIE ETAL,06/01/1988,1,13,-8,-8,NaN,1988


In [121]:
dummies = pd.get_dummies(clean_JFC_patents['NOS'])
clean_JFC_patents = pd.concat([clean_JFC_patents, dummies], axis=1)

dummies = pd.get_dummies(clean_JFC_patents['JUDGMENT'])
clean_JFC_patents = pd.concat([clean_JFC_patents, dummies], axis=1)

In [122]:
clean_JFC_patents = clean_JFC_patents.rename(columns={830: 'Patent', 840: 'Trademark'}).drop(columns='NOS')
clean_JFC_patents = clean_JFC_patents.rename(columns={-8: 'Missing',0: 'Missing', 1: 'Plaintiff', 2: 'Defendent',3: 'both', 4: 'Unkown' }).drop(columns='JUDGMENT')

In [123]:
clean_JFC_patents.loc[clean_JFC_patents['CLASSACT'] == -8, 'CLASSACT'] = 0

In [124]:
cases = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/cases.csv')
cases = cases[['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed', 'date_closed', 'jurisdictional_basis']]

In [125]:
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['DOCKET'].astype(str).str[:2] + "-cv-" + clean_JFC_patents['DOCKET'].astype(str).str[2:]
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['JURIS'].astype(str) + "-" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[0] + "/" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[1] + "/" + clean_JFC_patents['unique_identifier'].astype(str)
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['unique_identifier'] + "-" + clean_JFC_patents['TERMDATE'].astype(str)

#creating common unique identifier to match JFC dataset with
#creating common unique identifier to match JFC dataset with
cases['unique_identifier'] = cases['case_number'].astype(str).str.split(":", n=1).str[-1]
cases['unique_identifier'] = cases['date_filed'].astype(str).str.split("-").str[1] + "/" + cases['date_filed'].astype(str).str.split("-").str[2] + "/" +cases['unique_identifier'].astype(str)

for index, row in cases.iterrows():
    if row['jurisdictional_basis'] == "Federal Question":
        cases.loc[index, 'unique_identifier'] = "3-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Plaintiff":
        cases.loc[index, 'unique_identifier'] = "1-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Defendant":
        cases.loc[index, 'unique_identifier'] = "2-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "Diversity":
        cases.loc[index, 'unique_identifier'] = "4-" + row['unique_identifier']

cases['unique_identifier'] = cases['unique_identifier'] + "-" + cases['date_closed'].astype(str).str.split("-").str[1] + "/" + cases['date_closed'].astype(str).str.split("-").str[2] + "/" + cases['date_closed'].astype(str).str.split("-").str[0]

In [131]:
clean_JFC_patents

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,JURY,CLASSACT,DEMANDED,...,TAPEYEAR,Patent,Trademark,Missing,Missing,Plaintiff,Defendent,both,Unkown,unique_identifier
42,0,90,1,8203478,12/08/1982,12/01/1982,3,-8,0,-8,...,1988,True,False,True,False,False,False,False,False,3-12/08/82-cv-03478-02/10/1988
47,0,90,1,8300001,01/03/1983,01/01/1983,3,-8,0,-8,...,1988,True,False,False,False,False,True,False,False,3-01/03/83-cv-00001-02/17/1988
188,0,90,1,8403233,10/22/1984,10/01/1984,3,-8,0,150,...,1988,True,False,True,False,False,False,False,False,3-10/22/84-cv-03233-06/01/1988
241,0,90,1,8403720,12/06/1984,12/01/1984,3,-8,0,-8,...,1988,False,True,True,False,False,False,False,False,3-12/06/84-cv-03720-10/01/1987
447,0,90,1,8501100,04/08/1985,04/01/1985,3,-8,0,-8,...,1988,False,True,True,False,False,False,False,False,3-04/08/85-cv-01100-06/01/1988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10445328,11,3J,4,2100368,12/31/2021,12/01/2021,3,P,0,0,...,2099,True,False,True,False,False,False,False,False,3-12/31/21-cv-00368-01/01/1900
10445345,11,3J,4,2200291,12/12/2022,12/01/2022,3,B,0,0,...,2099,False,True,True,False,False,False,False,False,3-12/12/22-cv-00291-01/01/1900
10445359,11,3J,4,2300115,05/01/2023,05/01/2023,3,B,0,0,...,2099,False,True,True,False,False,False,False,False,3-05/01/23-cv-00115-01/01/1900
10445495,11,3J,4,2400155,07/22/2024,07/01/2024,3,P,0,0,...,2025,False,True,False,False,False,False,False,True,3-07/22/24-cv-00155-10/23/2024


In [127]:
cases.head()

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier
0,78028.0,4:92-cv-00055,"Melba Creek Mining, Inc. et al v. Sykes et al",No cause code entered,1992-12-22,1993-02-16,Federal Question,3-12/22/92-cv-00055-02/16/1993
1,75332.0,2:10-cv-00601,Bowhall v. Johnson & Johnson Inc. et al (MAG+),15:1126 Patent Infringement,2010-07-14,2010-10-12,Federal Question,3-07/14/10-cv-00601-10/12/2010
2,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998
3,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999
4,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018


In [128]:
merged = pd.merge(cases, clean_JFC_patents, on='unique_identifier', how='inner')

In [138]:
merged

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier,CIRCUIT,DISTRICT,...,STATUSCD,TAPEYEAR,Patent,Trademark,Missing,Missing,Plaintiff,Defendent,both,Unkown
0,75332.0,2:10-cv-00601,Bowhall v. Johnson & Johnson Inc. et al (MAG+),15:1126 Patent Infringement,2010-07-14,2010-10-12,Federal Question,3-07/14/10-cv-00601-10/12/2010,11,27,...,L,2011,True,False,True,False,False,False,False,False
1,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998,11,26,...,NaN,1998,True,False,True,False,False,False,False,False
2,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999,11,28,...,NaN,1999,True,False,True,False,False,False,False,False
3,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018,8,60,...,L,2018,True,False,False,True,False,False,False,False
4,30787.0,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al",15:1125 Trademark Infringement (Lanham Act),1999-04-09,1999-12-16,Federal Question,3-04/09/99-cv-00674-12/16/1999,9,70,...,NaN,2000,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57545,28883.0,2:95-cv-00217,Wyoming Woolens v. Acorn Products Co,28:1338 Patent Infringement,1995-09-29,1995-10-06,Federal Question,3-09/29/95-cv-00217-10/06/1995,10,89,...,NaN,1996,True,False,True,False,False,False,False,False
57546,101854.0,2:19-cv-00119,CUCU Inc v. Slickwraps Inc,35:145 Patent Infringement,2019-06-12,2019-10-10,Federal Question,3-06/12/19-cv-00119-10/10/2019,10,89,...,L,2020,True,False,False,True,False,False,False,False
57547,29713.0,2:97-cv-00019,Chey Land & Cattle v. Crown Cork & Seal Co,35:271 Patent Infringement,1997-01-24,1997-07-25,Federal Question,3-01/24/97-cv-00019-07/25/1997,10,89,...,NaN,1997,True,False,True,False,False,False,False,False
57548,39890.0,2:11-cv-00352,Weir Minerals Australia Ltd v. Megatrol Inc et al,35:271 Patent Infringement,2011-11-10,2012-11-15,Federal Question,3-11/10/11-cv-00352-11/15/2012,10,89,...,L,2013,True,False,False,True,False,False,False,False
